In [1]:
import numpy as np

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, Compose
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.datasets.AerialDroneSemanticSegmentationDataset import (
    AerialDroneSemanticSegmentationDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

d:\__repos\aerial_segmentation\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 1
SEED = 42
AERIAL_DRONE_DATASET_PATH = (
    "data/AerialDroneSemanticSegmentationDataset/dataset/semantic_drone_dataset"
)
IMAGE_SIZE = 576

In [5]:
train_dataset = AerialDroneSemanticSegmentationDataset(
    AERIAL_DRONE_DATASET_PATH,
    transforms=[Compose([Resize(IMAGE_SIZE), ResizeToDivisibleBy32()])],
)
print(len(train_dataset))

400


## Sanity check data

In [6]:
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [7]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([1, 3, 576, 864])
torch.Size([1, 1, 576, 864])


In [8]:
to_pil_transform = transforms.ToPILImage()

In [9]:
img = to_pil_transform(images.squeeze())

In [10]:
# img.show()

In [11]:
msk = to_pil_transform(masks.squeeze())

In [12]:
# msk.show()

## Run example model on single image

In [30]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=23,  # model output channels (number of classes in your dataset)
    activation="softmax",
).to(device)

In [31]:
baseline_model = BaselineModel(classes=23).to(device)

In [32]:
torch.cuda.empty_cache()

In [33]:
model.eval()
with torch.no_grad():
    for images, masks in train_loader:
        print(images.shape)
        print(masks.shape)
        break
    output = model(images.to(device))
    # output = (output > 0.5).float()

    output_baseline = baseline_model(images.to(device))
    # output_baseline = (output_baseline > 0.5).float()

torch.Size([1, 3, 576, 864])
torch.Size([1, 1, 576, 864])


d:\__repos\aerial_segmentation\venv\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [34]:
print(output.shape)

torch.Size([1, 23, 576, 864])


In [35]:
print(output_baseline.shape)

torch.Size([1, 23, 576, 864])


In [36]:
print(output.shape)
print(output_baseline.shape)

torch.Size([1, 23, 576, 864])
torch.Size([1, 23, 576, 864])


In [37]:
output = torch.argmax(output, dim=1)
output_baseline = torch.argmax(output_baseline, dim=1)

In [38]:
print(output.shape)
print(output_baseline.shape)

torch.Size([1, 576, 864])
torch.Size([1, 576, 864])


In [39]:
evaluate_result(output.squeeze(), masks.squeeze(), mode="multiclass", num_classes=23)

{'iou': tensor(0.0007),
 'f1': tensor(0.0014),
 'accuracy': tensor(0.9132),
 'recall': tensor(0.0014)}

In [40]:
evaluate_result(
    output_baseline.squeeze(), masks.squeeze(), mode="multiclass", num_classes=23
)

{'iou': tensor(0.0221),
 'f1': tensor(0.0432),
 'accuracy': tensor(0.9168),
 'recall': tensor(0.0432)}

## Run example model on dataset

In [41]:
model.eval()
metrics_dict = {
    "iou": [],
    "f1": [],
    "accuracy": [],
    "recall": [],
}

In [42]:
with torch.no_grad():
    for images, masks in train_loader:
        output = model(images.to(device))
        output = torch.argmax(output, dim=1)

        iter_metrics = evaluate_result(
            output.squeeze(), masks.squeeze(), mode="multiclass", num_classes=23
        )
        for key in metrics_dict.keys():
            metrics_dict[key].append(iter_metrics[key])

In [43]:
print(metrics_dict)

{'iou': [tensor(0.0004), tensor(0.0001), tensor(0.0005), tensor(0.0007), tensor(0.0008), tensor(0.0003), tensor(0.0011), tensor(0.0145), tensor(0.0314), tensor(0.0004), tensor(0.0002), tensor(0.0007), tensor(1.7080e-05), tensor(0.0125), tensor(0.0011), tensor(0.0005), tensor(0.0003), tensor(0.0002), tensor(0.0002), tensor(0.0007), tensor(0.0006), tensor(6.5309e-05), tensor(0.0002), tensor(0.0003), tensor(0.0002), tensor(0.0004), tensor(0.0002), tensor(0.0003), tensor(0.0006), tensor(0.0004), tensor(0.0004), tensor(6.3300e-05), tensor(0.0200), tensor(0.0002), tensor(0.0007), tensor(0.0002), tensor(0.0003), tensor(0.0069), tensor(0.0004), tensor(0.0002), tensor(0.0004), tensor(0.0007), tensor(0.0005), tensor(0.0005), tensor(0.0002), tensor(0.0002), tensor(0.0005), tensor(0.0003), tensor(0.0004), tensor(0.0001), tensor(0.0005), tensor(0.0007), tensor(0.0003), tensor(0.0003), tensor(0.0003), tensor(0.0002), tensor(0.0438), tensor(0.0004), tensor(2.0094e-05), tensor(0.0128), tensor(0.0087),

In [44]:
print("Mean metrics")
for key in metrics_dict.keys():
    print(key, np.mean(metrics_dict[key]))

Mean metrics
iou 0.004345709
f1 0.008359963
accuracy 0.91377044
recall 0.008359963
